In [2]:
import xarray as xr
import xesmf
import matplotlib.pyplot as plt
import numpy as np
import subprocess as sp
import os
import glob
import cartopy.crs as ccrs

IO_exp_dir = "/home/nma/HDD2/vinay/archives/IOMOM5Op/"
exp_dir = '/home/nma/mom/MOM6dev/prepro/obc2/'
dom_file = "/home/nma/mom/MOM6dev/exps/regional2/INPUT/bob_grid.nc"


## IOexpo U and V datasets

In [22]:
uu = xr.open_mfdataset(IO_exp_dir+"*.ocean_uvel.nc")
vv = xr.open_mfdataset(IO_exp_dir+"*.ocean_vvel.nc")

fin_mod_dat = xr.merge([uu,vv])


<xarray.Dataset>
Dimensions:         (xu_ocean: 360, yu_ocean: 240, st_ocean: 58, st_edges_ocean: 59, time: 1122, nv: 2)
Coordinates:
  * xu_ocean        (xu_ocean) float64 30.12 30.38 30.62 ... 119.4 119.6 119.9
  * yu_ocean        (yu_ocean) float64 -29.88 -29.62 -29.38 ... 29.62 29.88
  * st_ocean        (st_ocean) float64 0.5 1.5 2.5 ... 3.625e+03 4.3e+03 5e+03
  * st_edges_ocean  (st_edges_ocean) float64 0.0 1.0 2.0 ... 4.65e+03 5.35e+03
  * time            (time) object 2012-04-03 12:00:00 ... 2015-04-29 12:00:00
  * nv              (nv) float64 1.0 2.0
Data variables:
    u               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(515, 58, 240, 360), meta=np.ndarray>
    average_T1      (time) datetime64[ns] dask.array<chunksize=(515,), meta=np.ndarray>
    average_T2      (time) datetime64[ns] dask.array<chunksize=(515,), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(515,), meta=np.ndarray>
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(515, 2), meta=np.ndarray>
    v               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(515, 58, 240, 360), meta=np.ndarray>
Attributes:
    filename:   ocean_uvel.nc
    title:      IOMV6d : Inter-annual  QuikSCAT Run
    grid_type:  regular
    grid_tile:  N/A

## Domain Configs

Here there are two grids
1. regional2 model supergrid (The grid of simulation model)
2. Grid system of the boundary condition input file

In [35]:
grid_ori = xr.open_dataset(dom_file)

grid["ds"]= xr.open_dataset(dom_file,decode_times=False)
grid['ds']=grid['ds'].drop_dims(['ny','nx'])
grid['ds']=grid['ds'].drop_vars(['tile'])
grid['nyp']=grid['ds'].nyp.data[-1]+1
grid['nxp']=grid['ds'].nxp.data[-1]+1
nxp=grid['nxp'];nyp=grid['nyp']
grid['h'] = grid['ds'].isel(nxp=slice(1,nxp+1,2),nyp=slice(1,nyp+1,2))
grid['q'] = grid['ds'].isel(nxp=slice(2,nxp+1,2),nyp=slice(2,nyp+1,2))
grid['Cu'] = grid['ds'].isel(nxp=slice(0,nxp+1,2),nyp=slice(1,nyp+1,2))
grid['Cv'] = grid['ds'].isel(nxp=slice(1,nxp+1,2),nyp=slice(0,nyp+1,2))



## Making new grid system for model input data

In [50]:
u_only = uu.u
sam_op_dir = xr.open_dataset("
 
sample_model = 

<xarray.DataArray 'u' (time: 1122, st_ocean: 58, yu_ocean: 240, xu_ocean: 360)>
dask.array<concatenate, shape=(1122, 58, 240, 360), dtype=float32, chunksize=(607, 58, 240, 360), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 30.12 30.38 30.62 30.88 ... 119.4 119.6 119.9
  * yu_ocean  (yu_ocean) float64 -29.88 -29.62 -29.38 ... 29.38 29.62 29.88
  * st_ocean  (st_ocean) float64 0.5 1.5 2.5 3.5 ... 3.625e+03 4.3e+03 5e+03
  * time      (time) object 2012-04-03 12:00:00 ... 2015-04-29 12:00:00
Attributes:
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    standard_name:  sea_water_x_velocity

## Setting up tracers

In [47]:

fields=[{'temp':'h'},{'salt':'h'},{'ssh':'h'},{'u':'Cu'},{'v':'Cv'}]

ds=fin_mod_dat
tracer_list=[];uv_list=[]
for f in fields:
    for fnam,val in zip(f.keys(),f.values()):
        if val=='h':tracer_list.append(fnam)
        if val=='u':uv_list.append(fnam)
        if val=='v':uv_list.append(fnam)
        
    ds_tr = xr.merge([ds, grid['h']])
    ds_u= xr.merge([ds,grid['Cu']])
    ds_v= xr.merge([ds,grid['Cv']])



<xarray.Dataset>
Dimensions:         (xu_ocean: 360, yu_ocean: 240, st_ocean: 58, st_edges_ocean: 59, time: 1122, nv: 2, nyp: 192, nxp: 263)
Coordinates:
  * xu_ocean        (xu_ocean) float64 30.12 30.38 30.62 ... 119.4 119.6 119.9
  * yu_ocean        (yu_ocean) float64 -29.88 -29.62 -29.38 ... 29.62 29.88
  * st_ocean        (st_ocean) float64 0.5 1.5 2.5 ... 3.625e+03 4.3e+03 5e+03
  * st_edges_ocean  (st_edges_ocean) float64 0.0 1.0 2.0 ... 4.65e+03 5.35e+03
  * time            (time) object 2012-04-03 12:00:00 ... 2015-04-29 12:00:00
  * nv              (nv) float64 1.0 2.0
Dimensions without coordinates: nyp, nxp
Data variables:
    u               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(515, 58, 240, 360), meta=np.ndarray>
    average_T1      (time) datetime64[ns] dask.array<chunksize=(515,), meta=np.ndarray>
    average_T2      (time) datetime64[ns] dask.array<chunksize=(515,), meta=np.ndarray>
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(515,), meta=np.ndarray>
    time_bounds     (time, nv) timedelta64[ns] dask.array<chunksize=(515, 2), meta=np.ndarray>
    v               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(515, 58, 240, 360), meta=np.ndarray>
    x               (nyp, nxp) float64 ...
    y               (nyp, nxp) float64 ...
    angle_dx        (nyp, nxp) float64 ...
    arcx            |S255 ...
Attributes:
    filename:   ocean_uvel.nc
    title:      IOMV6d : Inter-annual  QuikSCAT Run
    grid_type:  regular
    grid_tile:  N/A

In [46]:
nxp=ds_u.nxp[-1].data+1;nyp=ds_v.nyp[-1].data+1
#upper-right q points
u_q=0.5*(ds_u.u+ds_u.u.roll(roll_coords='yh',yh=-1)).isel(xq=slice(1,nxp))
#upper-right q points
v_q=0.5*(ds_v.v+ds_v.v.roll(roll_coords='xh',xh=-1)).isel(yq=slice(1,nyp))
ds_uvq = xr.Dataset({'u':u_q,'v':v_q},coords={'time':ds_u.time,'lon':parent_grid['q'].x,'lat':parent_grid['q'].y,'angle_dx':parent_grid['q'].angle_dx})


ValueError: dimensions ['yh'] do not exist